# TODOs
- [ ] Runtimes with various input size and different optimization flags
> Create a timing infrastructure to get runtimes for a suitable set of input sizes (and inputs). Push sizes to the limit (i.e., until the execution takes minutes or even hours). Consider different compiler flags (but always with optimization).
- [ ] Cost Analysis (float ops, float comp, int ops, int comp)
> Cost analysis: Select a cost measure (usually flop count but may include div, sin, ...) and determine the cost. Briefly say how you did it (e.g., counted in code, had to instrument parts, ...).
- [ ] Performance Plot
> Create a performance plot. Put it as plot0 (or plot0a, plot0b, ...) into svn in the same directory as this document. As you perform optimizations later, include one line for each relevant optimization.
- [ ] Bottlenecks by profiling tool
> Identify performance bottlenecks in your code by profiling. This will likely involve instrumenting your code (standard profilers may not be fine grain enough). Focus on those parts when optimizing. The bottlenecks may change during optimization. Understand the cost and performance of the bottlenecks to set expectations right.

In [1]:
import re
import numpy as np    
import pandas as pd

general_log = "./general_metrics.txt"
cache_log = './cache_log.txt'
flops_log = './flops_log.txt'

In [2]:
def extract(log, metric):
    res = []
    with open(log, 'r') as f:
        file = f.read()
    try:
        if metric=='runtime(sec)':
            res = re.findall("(.*) seconds time elapsed", file)
        elif metric=='cc_int_ops':
            res = re.findall("cc_int_ops:(.*), cc_int_comp", file)
        elif metric=='cc_int_comp':
            res = re.findall("cc_int_comp:(.*), cc_fl_ops", file)
        elif metric=='cc_fl_ops':
            res = re.findall("cc_fl_ops(.*), cc_fl_comp", file)
        elif metric=='cc_fl_comp':
            res = re.findall("cc_fl_comp:(.*)", file)
        elif metric=='ll_int_ops':
            res = re.findall("ll_int_ops:(.*), ll_int_comp:", file)
        elif metric=='ll_int_comp':
            res = re.findall("ll_int_comp:(.*)", file)
        elif metric=='flops':
            res = re.findall("#fl ops: (.*)", file)
        elif metric=='flcomp':
            res = re.findall("#fl comparisons: (.*)", file)
        elif metric=='intops':
            res = re.findall("#int ops: (.*)", file)
        elif metric=='intcomp':
            res = re.findall("#int comparisons: (.*)", file)
        elif metric=='cycles':
            res = re.findall("#cycles: (.*)", file)
        elif metric=='flops/cycles':
            res = re.findall("(.*)flops / cycles", file)
        elif metric=='instructions':
            res = re.findall("(.*)   instructions:u", file)
        elif metric=='branches':
            res = re.findall("(.*)     branches:u", file)
        elif metric=='branch-misses':
            res = re.findall("branch-misses:u           #(.*) of all branches", file)
        elif metric=='page-faults':
            res = re.findall("(.*)  page-faults:u", file)
        elif metric=='cycles(perf)':
            res = re.findall("(.*)      cycles:u", file)
        elif metric=='flops(perf)':
            res = re.findall("(.*) r5301c7", file)
        elif metric=='L1-dcache-loads':
            res = re.findall("(.*)L1-dcache-loads:u", file)
        elif metric=='L1-dcache-load-misses':
            res = re.findall("L1-dcache-load-misses:u   #(.*)\% ", file)
        elif metric=='LLC-loads':
            res = re.findall("(.*)LLC-loads:u", file)
        elif metric=='LLC-load-misses':
            res = re.findall("LLC-load-misses:u         #(.*)\% ", file)
    except Exception:
        return 
    return res  

def init_table(vertice, edge):
    ve = []
    for v in vertice:
        for e in edge:
            if e < v*10 and e >= v:
                ve.append([v,e])
    return pd.DataFrame(ve, columns=['#V', '#E'])

def update_table(df, log, data, header):
    for i, x in enumerate(df['#V']):
        for j, y in enumerate(df['#E']):
            for l in header:
                data[l] = extract(log, l)
                data[l] = [float(x.replace(',', '').strip().strip('%')) for x in data[l]]
                if len(data[l])==len(df):
                    df[l] = data[l]
                else:
                    print(f"fail to add {l} to the table.")

In [3]:
from itertools import product

label=['runtime(sec)',
       'flops','flcomp','intops','intcomp', 'flops/cycles',
      #'cc_int_ops', 'cc_int_comp', 'cc_fl_ops', 'cc_fl_comp', 'll_int_ops', 'll_int_comp',
      'instructions', 'cycles', 'cycles(perf)', 
      'branches', 'branch-misses', 'page-faults']
cache=['L1-dcache-loads','L1-dcache-load-misses','LLC-loads','LLC-load-misses',
      # 'LLC-stores','LLC-store-misses','L1-dcache-stores', 'dTLB-loads','dTLB-load-misses','dTLB-stores','dTLB-store-misses',
      # 'iTLB-loads', 'iTLB-load-misses'
       ]
flops=['flops(perf)']

vertice = [10,100,1000,2000,5000,8000,10000]
edge = [10,100,500,1000,2000,5000,8000,10000,20000,50000]

In [4]:
# results with flag -O3
data = dict()
res = init_table(vertice, edge)
res = res[:-1]
update_table(res, general_log, data, label)
update_table(res, flops_log, data, cache)
#res.to_excel('result.xlsx', sheet_name='sheet1', index=False) 
res

,#V,#E,runtime(sec),flops,flcomp,intops,intcomp,flops/cycles,instructions,cycles,cycles(perf),branches,branch-misses,page-faults,L1-dcache-loads,L1-dcache-load-misses,LLC-loads,LLC-load-misses
0,10,10,0.010036,71.0,15.0,50.0,126.0,0.006646,4.373400e+07,3.942090e+04,2.740691e+07,7.738926e+06,0.70,420.0,1.303553e+07,0.40,1.610300e+04,1.26
1,100,100,0.160082,1278.0,275.0,776.0,2007.0,0.005476,1.032890e+09,7.918740e+05,5.767099e+08,1.753785e+08,0.32,430.0,3.506302e+08,0.32,1.119740e+05,0.21
2,100,500,0.686664,12156.0,7819.0,13649.0,36298.0,0.018763,4.696919e+09,3.726560e+06,2.490287e+09,7.883183e+08,0.46,451.0,1.680453e+09,0.23,2.070440e+05,0.09
3,1000,1000,8.242616,13296.0,2851.0,8002.0,20783.0,0.000962,6.437297e+10,4.671830e+07,3.004798e+10,1.085944e+10,0.09,495.0,2.298848e+10,2.34,9.378200e+05,0.15
4,1000,2000,15.706050,68545.0,31960.0,58659.0,160084.0,0.003566,1.300121e+11,8.951890e+07,5.783470e+10,2.209797e+10,0.11,547.0,4.646131e+10,1.89,2.003659e+06,0.13
5,1000,5000,38.363303,195773.0,132635.0,228933.0,594329.0,0.005205,3.233033e+11,2.212810e+08,1.408518e+11,5.530765e+10,0.13,802.0,1.157592e+11,1.44,2.166216e+07,1.27
6,1000,8000,62.502320,302344.0,240501.0,412317.0,1038420.0,0.005537,5.187104e+11,3.600600e+08,2.283565e+11,8.882685e+10,0.15,976.0,1.858596e+11,1.32,6.007135e+07,0.66
7,2000,2000,29.968099,26970.0,5765.0,16054.0,41925.0,0.000535,2.505545e+11,1.696740e+08,1.094734e+11,4.202926e+10,0.05,567.0,9.028668e+10,3.02,1.234181e+07,0.22
8,2000,5000,74.865500,202701.0,109035.0,192922.0,516866.0,0.002401,6.230879e+11,4.254190e+08,2.724296e+11,1.062544e+11,0.07,804.0,2.235960e+11,2.00,1.510334e+08,0.39
9,2000,8000,117.836783,347583.0,223132.0,386353.0,1009192.0,0.002919,9.930165e+11,6.735980e+08,4.296805e+11,1.700701e+11,0.08,1097.0,3.559588e+11,1.69,2.941797e+08,1.11


In [5]:
import plotly.express as px
import plotly.graph_objects as go

num_font = dict(size=22,color='#000000')
default_font = dict(family="Helvetica Neue",size=20,color='#000000')
#colors=px.colors.sequential.deep[2:]  # Candidate: Prism
colors=px.colors.sequential.Cividis[::-1][1:]
marker_symbols=['circle','square','diamond','cross','triangle-up-open','pentagon','hexagram','star','hourglass', 'asterisk']

In [6]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["flops/cycles"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True, rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_flops.pdf")

In [7]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["flcomp"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_flcomp.pdf")

In [8]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["cycles"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_cycles.pdf") #similar graph for runtime, 

In [9]:
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["page-faults"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_page-faults.pdf")

In [10]:
# branch-misses
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["branch-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_branch_miss.pdf")

In [11]:
# runtime
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["runtime(sec)"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_runtime.pdf") # similar graph for cycles, 

In [12]:
# L1-dcache-load-misses	
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["L1-dcache-load-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_L1-dcache-load-misses.pdf")

In [13]:
# LLC-load-misses
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["LLC-load-misses"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_LLC-load-misses.pdf")

In [14]:
# LLC-load vs L1-dcache-loads
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=same_e["L1-dcache-loads"]/same_e["LLC-loads"], name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_L1toLLC.pdf")

In [15]:
# operation intensity: instruction/memory
fig = go.Figure()
ploted_edge=[100,1000,2000,5000,8000,10000,20000]
for idx, e in enumerate(ploted_edge):
    same_e = res.loc[res['#E'] == e]
    man_instr = same_e["flops"]+same_e["flcomp"]+same_e["intops"]+same_e["intcomp"]
    operation = same_e["instructions"]/same_e["L1-dcache-loads"]
    fig.add_trace(go.Scatter(x=same_e["#V"], y=operation, name=f"{e}",
                             mode='lines+markers', marker=dict(symbol=marker_symbols[idx],size=10),
                             line=dict(color=colors[idx], width=3)))

fig.update_layout(xaxis=dict(showline=True, showgrid=False, rangemode="tozero",
                             linecolor='rgb(0, 0, 0)', linewidth=2,
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,rangemode="tozero",
                             showline=False),
                  showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_oi.pdf")

In [16]:
# flops/cycle vs oi
fig = go.Figure()

fig.add_trace(go.Scatter(x=res["flops"]/res["L1-dcache-loads"], y=res["flops/cycles"], 
                         mode='lines+markers',marker=dict(symbol=marker_symbols[3],size=5),
                         line=dict(color=colors[3], width=1)))

fig.update_layout(xaxis=dict(showline=True, showgrid=True,
                             linecolor='rgb(0, 0, 0)', linewidth=2,rangemode="nonnegative",
                             tickwidth=3, tickfont=num_font, ticks='outside'),
                  yaxis=dict(tickfont=num_font, zeroline=False, showticklabels=True,
                             rangemode="nonnegative",
                             showline=True),
                  #showlegend=True,legend=dict(traceorder="reversed"),
                  margin=dict(autoexpand=True, l=40, r=20, t=40),
                  plot_bgcolor="#eeeeee"
                  )
fig.show()
fig.write_image("baseline_roofline.pdf")